In [81]:
import numpy as np 

In [138]:
# this activation functions classes 
class sigmoid : 
    def forword(self , inputs) : # sigmoid
        return 1 / (1+np.exp(-inputs)) 
    def backword(self , inputs ): # sigmoid derivative
         return inputs * (1-inputs)
class relu : 
    def forword(self ,inputs) : # relu
        mask = (inputs > 0 )
        return inputs * mask
    def backword(self , inputs ): # relu derivative
        mask = (inputs > 0)
        return inputs * mask
class tanh : 
    def forword(self, inputs) : # tanh 
        return np.tanh(inputs)
    def backword(self, inputs ) : # tanh derivative 
        return 1 - (inputs**2)

In [139]:
# this optimizers classes  
class gradient_decent : # this normal gradient decent 
    def __init__(self, lr = .001 ): 
        self.lr = lr # learning rate
    def back_prop(self , dAL, objects , l , lambd , m  , activation ) : 
        '''
            objects : it,s contain all weights and baises and activatoins from the original model 
            dAl : the gradients of the loss with respect to weights 
            l : number of layers 
            lambd : regularization prameter 
            m : training examples 
            activation : the activation function 
        '''
        # compute the gradients for the weights and biases for the output layer
        vars(self)[f'dZ_{l}'] = dAL * sigmoid.backword(objects[f'A_{l}']) 
        vars(self)[f'dW_{l}'] =1. /m * np.dot(  vars(self)[f'dZ_{l}'] , objects[f'A_{l-1}'].T) + (lambd / m ) * objects[f'W_{l}'] 
        vars(self)[f'db_{l}'] = np.sum(vars(self)[f'dZ_{l}'] , axis = 1, keepdims =True) / m
        vars(self)[f'dA_{l-1}'] = np.dot(objects[f'W_{l}'].T ,vars(self)[f'dZ_{l}'] ) 
        for i in reversed(range(1 , l)) : # iteraite backword throw the hidden layers 
        # compute the gradients for weights and biases for the hidden layers  
            vars(self)[f'dZ_{i}'] = vars(self)[f'dA_{i}'] * activation.backword(objects[f'A_{i}'])
            vars(self)[f'dW_{i}'] =1. / m *  np.dot( vars(self)[f'dZ_{i}'] ,objects[f'A_{i-1}'].T) + (lambd / m ) * objects[f'W_{i}']
            vars(self)[f'db_{i}'] = np.sum(vars(self)[f'dZ_{i}'] , keepdims= True , axis =1) /m
            vars(self)[f'dA_{i-1}'] = np.dot(objects[f'W_{i}'].T ,vars(self)[f'dZ_{i}'] )
      
      # update the weights and biases
        for i in range(1, l+1) : 
            objects[f'W_{i}'] -= self.lr * vars(self)[f'dW_{i}']
            objects[f'b_{i}'] -= self.lr * vars(self)[f'db_{i}']
class Adam  :  
    
    def __init__(self, beta1 = .9  ,lr = .001, beta2 = .999 ,  epsilon= 1e-8) : 
        
        self.lr = lr  # learning rate 
        self.beta1 = beta1 
        self.beta2 = beta2
        self.epsilon = epsilon
    def build(self , objects , l ) : 
        for i in range( 1 , l+1 ): 
            # init the weights for the optimizer 
            vars(self)[f'vdW_{i}'] = np.zeros((objects[f'W_{i}'].shape))
            vars(self)[f'vdb_{i}'] = np.zeros((objects[f'b_{i}'].shape))
            vars(self)[f'sdW_{i}'] = np.zeros((objects[f'W_{i}'].shape))
            vars(self)[f'sdb_{i}'] = np.zeros((objects[f'b_{i}'].shape))
    def back_prop(self , dAL, objects , l , lambd , m  , activation ) :
         '''
            objects : it,s contain all weights and baises and activatoins from the original model 
            dAl : the gradients of the loss with respect to weights 
            l : number of layers 
            lambd : regularization prameter 
            m : training examples 
            activation : the activation function 
        '''
        # call method build to init the weights 
        self.build(objects , l )
        # compute the gradients of the output layer 
        vars(self)[f'dZ_{l}'] = dAL * sigmoid.backword(objects[f'A_{l}']) 
        vars(self)[f'dW_{l}'] =1. /m * np.dot(  vars(self)[f'dZ_{l}'] , objects[f'A_{l-1}'].T) + (lambd / m ) * objects[f'W_{l}'] 
        vars(self)[f'db_{l}'] = np.sum(vars(self)[f'dZ_{l}'] , axis = 1, keepdims =True) / m
        vars(self)[f'vdW_{l}'] = self.beta1 * vars(self)[f'vdW_{l}'] + (1- self.beta1) * vars(self)[f'dW_{l}']
        vars(self)[f'vdb_{l}'] = self.beta1 * vars(self)[f'vdb_{l}'] + (1- self.beta1) * vars(self)[f'db_{l}']
        vars(self)[f'sdW_{l}'] = self.beta2 * vars(self)[f'sdW_{l}'] + (1-self.beta2) * (vars(self)[f'dW_{l}'] ** 2)
        vars(self)[f'sdb_{l}'] = self.beta2 * vars(self)[f'sdb_{l}'] + (1-self.beta2) * (vars(self)[f'db_{l}'] ** 2)
        vars(self)[f'dA_{l-1}'] = np.dot(objects[f'W_{l}'].T ,vars(self)[f'dZ_{l}'] ) 
        for i in reversed(range(1 , l)) : # iteraite backword throw the hidden layers 
        # compute the gradients for weights and biases for the hidden layers  
            vars(self)[f'dZ_{i}'] = vars(self)[f'dA_{i}'] * activation.backword(objects[f'A_{i}'])
            vars(self)[f'dW_{i}'] =1. / m *  np.dot( vars(self)[f'dZ_{i}'] ,objects[f'A_{i-1}'].T) + (lambd / m ) * objects[f'W_{i}']
            vars(self)[f'db_{i}'] = np.sum(vars(self)[f'dZ_{i}'] , keepdims= True , axis =1) /m
            vars(self)[f'vdW_{i}'] = self.beta1 * vars(self)[f'vdW_{i}'] + (1- self.beta1) * vars(self)[f'dW_{i}']
            vars(self)[f'vdb_{i}'] = self.beta1 * vars(self)[f'vdb_{i}'] + (1- self.beta1) * vars(self)[f'db_{i}']
            vars(self)[f'sdW_{i}'] = self.beta2 * vars(self)[f'sdW_{i}'] + (1-self.beta2) * (vars(self)[f'dW_{i}'] ** 2)
            vars(self)[f'sdb_{i}'] = self.beta2 * vars(self)[f'sdb_{i}'] + (1-self.beta2) * (vars(self)[f'db_{i}'] ** 2)
            vars(self)[f'dA_{i-1}'] = np.dot(objects[f'W_{i}'].T ,vars(self)[f'dZ_{i}'] )
      
      # update the weights and biases
        for i in range(1, l+1) : 
            objects[f'W_{i}'] -= self.lr * (vars(self)[f'vdW_{i}']/ (np.sqrt(vars(self)[f'sdW_{i}'] + self.epsilon)))
            objects[f'b_{i}'] -= self.lr * (vars(self)[f'vdb_{i}']/ (np.sqrt(vars(self)[f'sdb_{i}'] + self.epsilon)))
class rmsprop  :  
    def __init__(self, beta = .9  ,lr = .001  ,epsilon= 1e-8) : 
        self.lr = lr 
        self.beta1 = beta1
        self.epsilon = epsilon
    def build(self , objects , l ) : 
        for i in range( 1 , l+1 ): 
            vars(self)[f'sdW_{i}'] = np.zeros((objects[f'W_{i}'].shape))
            vars(self)[f'sdb_{i}'] = np.zeros((objects[f'b_{i}'].shape))
    def back_prop(self , dAL, objects , l , lambd , m  , activation ) : 
         '''
            objects : it,s contain all weights and baises and activatoins from the original model 
            dAl : the gradients of the loss with respect to weights 
            l : number of layers 
            lambd : regularization prameter 
            m : training examples 
            activation : the activation function 
        '''
        # call method build 
        self.build(objects , l )
        
        vars(self)[f'dZ_{l}'] = dAL * sigmoid.backword(objects[f'A_{l}']) 
        vars(self)[f'dW_{l}'] =1. /m * np.dot(  vars(self)[f'dZ_{l}'] , objects[f'A_{l-1}'].T) + (lambd / m ) * objects[f'W_{l}'] 
        vars(self)[f'db_{l}'] = np.sum(vars(self)[f'dZ_{l}'] , axis = 1, keepdims =True) / m
        vars(self)[f'sdW_{l}'] = self.beta2 * vars(self)[f'sdW_{l}'] + (1-self.beta2) * (vars(self)[f'dW_{l}'] ** 2)
        vars(self)[f'sdb_{l}'] = self.beta2 * vars(self)[f'sdb_{l}'] + (1-self.beta2) * (vars(self)[f'db_{l}'] ** 2)
        vars(self)[f'dA_{l-1}'] = np.dot(objects[f'W_{l}'].T ,vars(self)[f'dZ_{l}'] ) 
        for i in reversed(range(1 , l)) : # iteraite backword throw the hidden layers 
        # compute the gradients for weights and biases for the hidden layers  
            vars(self)[f'dZ_{i}'] = vars(self)[f'dA_{i}'] * activation.backword(objects[f'A_{i}'])
            vars(self)[f'dW_{i}'] =1. / m *  np.dot( vars(self)[f'dZ_{i}'] ,objects[f'A_{i-1}'].T) + (lambd / m ) * objects[f'W_{i}']
            vars(self)[f'db_{i}'] = np.sum(vars(self)[f'dZ_{i}'] , keepdims= True , axis =1) /m
            vars(self)[f'sdW_{i}'] = self.beta2 * vars(self)[f'sdW_{i}'] + (1-self.beta2) * (vars(self)[f'dW_{i}'] ** 2)
            vars(self)[f'sdb_{i}'] = self.beta2 * vars(self)[f'sdb_{i}'] + (1-self.beta2) * (vars(self)[f'db_{i}'] ** 2)
            vars(self)[f'dA_{i-1}'] = np.dot(objects[f'W_{i}'].T ,vars(self)[f'dZ_{i}'] )
      
      # update the weights and biases
        for i in range(1, l+1) : 
            objects[f'W_{i}'] -= self.lr * (objects[f'dW_{i}']/ (np.sqrt(vars(self)[f'sdW_{i}'] + self.epsilon)))
            objects[f'b_{i}'] -= self.lr * (objects[f'db_{i}']/ (np.sqrt(vars(self)[f'sdb_{i}'] + self.epsilon)))
   
            
   
            

In [140]:
class deep_nn_model : 
    def __init__(self , l ,layer_dims , activation ):
        '''
        l : number of layers 
        layer_dims : list containing the hidden units for each hidden layer for example [num_features ,128 ,64,32 ]
        '''
        self.activation = activation
        self.l = l 
        self.layer_dims = layer_dims 
    def build(self): 
        # init the weights and baises 
        for i in range(1, self.l + 1) : 
            vars(self)[f'W_{i}'] = np.random.randn(self.layer_dims[i] ,self.layer_dims[i-1] ) * np.sqrt(1/self.layer_dims[i-1])
            vars(self)[f'b_{i}'] = np.zeros((self.layer_dims[i] , 1))
    def fit(self, X , Y   , lambd = .9 , num_iter = 2000) : 
        ''' 
        X : input data with shape(num_features , num_examples) 
        Y : input labels with shape(1 , num_examples)
        lambd : the regularization parameter 
        '''
        self.build()
        activation = self.activation
        losses = []
        m = X.shape[-1]
        # assign the X to activation 0 
        vars(self)[f'A_0'] = X
        for _ in range(num_iter): # compute the forword pass 
            for i in range(1,self.l) : 
                Z = np.dot(vars(self)[f'W_{i}'] , vars(self)[f'A_{i-1}']) + vars(self)[f'b_{i}'] 
                vars(self)[f'A_{i}'] = activation.forword(Z)
            lastZ = np.dot(vars(self)[f'W_{self.l}'] , vars(self)[f'A_{self.l-1}']) + vars(self)[f'b_{self.l}'] 
            vars(self)[f'A_{self.l}'] = sigmoid.forword(lastZ)
            # compute the loss 
            loss = - (np.dot(Y , np.log(vars(self)[f'A_{self.l}'].T) + np.dot((1-Y) , np.log((1-vars(self)[f'A_{self.l}']).T))) /m)
            l2_reg = 0
            # the l2 reqularization 
            for i in range(1 , self.l+1) : 
                l2_reg += np.sum(np.square(vars(self)[f'W_{i}'])) 
            l2_reg = lambd * l2_reg / (2 * m)
            loss = loss + l2_reg
            losses.append(loss)
            # the drevitve of the loss with respect the last activation 
            dAL = - (np.divide(Y, vars(self)[f'A_{self.l}' ]) - np.divide(1 -Y , 1 - vars(self)[f'A_{self.l}']))
            opt.back_prop(  dAL = dAL, objects = vars(self) ,  l = self.l , lambd = lambd  ,m =  m , activation = activation  )
            
    def predict(self  , X ) : 
        activation = self.activation
        vars(self)[f'AP_{0}'] = X 
        for i in range(1,self.l) : 
            ZP = np.dot(vars(self)[f'W_{i}'] , vars(self)[f'AP_{i-1}']) + vars(self)[f'b_{i}'] 
            vars(self)[f'AP_{i}'] = activation.forword(ZP)
        last_PZ = np.dot(vars(self)[f'W_{self.l}'] , vars(self)[f'AP_{self.l-1}']) + vars(self)[f'b_{self.l}'] 
        vars(self)[f'AP_{self.l}'] = sigmoid.forword(last_PZ)
        return vars(self)[f'AP_{self.l}']
    def evaluate(self , preds , true_y ) : 
        num_correct = 0 
        preds = (preds > .5) 
        for i in range(true_y.shape[0]) : 
             if preds[0][i] == true_y[i] : 
                num_correct += 1 
        return num_correct / true_y.shape[0]
        

In [141]:
# load the data 
import tensorflow 
(train_data , train_labels ) , (test_data , test_labels) = tensorflow.keras.datasets.mnist.load_data()
train_data , test_data = train_data / 255 , test_data /255
train_data = train_data.reshape((60000 , 28*28))
# take subset of the train data to train on because the dataset is so big
train_subset = train_data[:5000 , :]
train_subset_label = train_labels[:5000]

# because the mnist is multiclass problem and we want it to be binary we will change it 
# instead of predicting the number we will predict if this number is odd or even

# convert the label to binary
for i in range(5000) : 
  if train_subset_label[i] // 2 == 0 :
    train_subset_label[i] = 0
  else :
    train_subset_label[i] = 1
# take subset of the train data to test on because the dataset is so big
test_subset = train_data[5001 :6001 , :]  # i took the subset from the training data but you can take it from the test data
test_subset_label = train_labels[5001 : 6001]

# because the mnist is multiclass problem and we want it to be binary we will change it 
# instead of predicting the number we will predict if this number is odd or even

# convert the label to binary
for i in range(1000) : 
    if test_subset_label[i] // 2 == 0 :
        test_subset_label[i] = 0
    else :
        test_subset_label[i] = 1

In [142]:
sigmoid = sigmoid()
tanh = tanh()
opt = Adam()

In [143]:
model = deep_nn_model(3 , [784 , 64 ,32 ,1] , activation = tanh )

In [145]:
model.fit(train_subset.T, train_subset_label.T) 

In [133]:
preds = model.predict(test_subset.T)

In [134]:
acc = model.evaluate( preds  ,test_subset_label.T )